In [ ]:
import torch
import torchvision

In [ ]:
batch_size = 64
epochs = 10
lr = 0.001

In [ ]:
class Model(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = torch.nn.Linear(784, 128)
    self.fc2 = torch.nn.Linear(128, 10)

  def forward(self , x):
    x = x.reshape((x.shape[0], 784))
    x = self.fc1(x)
    x = torch.relu(x)
    x = torch.dropout(x, 0.2 , train=True)
    x = self.fc2(x)
    x = torch.softmax(x, dim=1)
    return x
    

In [ ]:
cudaDevice = torch.device("cuda")
model = Model()
model.to(cudaDevice)
model.train(True)

Model(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
transforms = torchvision.transforms.Compose([
       torchvision.transforms.ToTensor(),
       torchvision.transforms.Normalize((0),(1))                      
])

dataset = torchvision.datasets.MNIST("./dataset",download=True, transform=transforms, train=True)
trainData = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
lossFunction = torch.nn.CrossEntropyLoss()

In [ ]:
def accuracyCacculator(preds, lables):
  _, preds_max = torch.max(preds,1)
  acc = torch.sum(preds_max == lables , dtype=torch.float64)/len(preds)
  return acc

In [ ]:
for epoch in range(epochs):
  trainLoss = 0
  trainAccuracy = 0
  for images, labels in trainData:
    images = images.to(cudaDevice)
    labels = labels.to(cudaDevice)
    optimizer.zero_grad()

    predictions = model(images)
    loss = lossFunction(predictions,labels)
    loss.backward()
    optimizer.step()

    trainLoss += loss
    trainAccuracy += accuracyCacculator(predictions, labels)

  totalLoss = trainLoss/len(trainData)
  totalAccuracy = trainAccuracy/len(trainData)
  print(f"Epoch: {epoch +1}, Loss: {totalLoss}, Accuracy: {totalAccuracy}")

Epoch: 1, Loss: 1.607242465019226, Accuracy: 0.8811467217484008
Epoch: 2, Loss: 1.5359927415847778, Accuracy: 0.9336853678038379
Epoch: 3, Loss: 1.5203144550323486, Accuracy: 0.9465951492537313
Epoch: 4, Loss: 1.510363221168518, Accuracy: 0.9559401652452025
Epoch: 5, Loss: 1.5041923522949219, Accuracy: 0.960920842217484
Epoch: 6, Loss: 1.499473214149475, Accuracy: 0.9652185501066098
Epoch: 7, Loss: 1.4957292079925537, Accuracy: 0.96875
Epoch: 8, Loss: 1.4930495023727417, Accuracy: 0.9710820895522387
Epoch: 9, Loss: 1.491082787513733, Accuracy: 0.972581289978678
Epoch: 10, Loss: 1.489253282546997, Accuracy: 0.9742470682302772


In [ ]:
# Inference

